In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install captum
!pip install torchmetrics
%pip install 'drive/MyDrive/encoder_attribution_priors/.'

Processing ./drive/MyDrive/encoder_attribution_priors
  Preparing metadata (setup.py) ... done
  Created wheel for lfxai: filename=lfxai-0.1.1-py3-none-any.whl size=30915 sha256=a876edda75cc35a8319b0d4ac49919fc3b8348608b3ee409142009155ecba94e
  Stored in directory: /tmp/pip-ephem-wheel-cache-xlkeo0g5/wheels/7b/00/41/6fc986ab56840bbe22a1da3b92c41c9d55e2dcd851163270b7
Successfully built lfxai
  Attempting uninstall: lfxai
    Found existing installation: lfxai 0.1.1
    Uninstalling lfxai-0.1.1:
      Successfully uninstalled lfxai-0.1.1


In [2]:
import argparse
import csv
import itertools
import logging
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torchvision
from captum.attr import GradientShap, IntegratedGradients, Saliency
from scipy.stats import spearmanr
from torch.utils.data import DataLoader, RandomSampler, Subset
from torchvision import transforms

from lfxai.explanations.examples import (
    InfluenceFunctions,
    NearestNeighbours,
    SimplEx,
    TracIn,
)
from lfxai.explanations.features import attribute_auxiliary, attribute_individual_dim, tensor_attribution, attribute_training
from lfxai.models.images import (
    VAE,
    AutoEncoderMnist,
    ClassifierMnist,
    DecoderBurgess,
    DecoderMnist,
    EncoderBurgess,
    EncoderMnist,
)
from lfxai.models.losses import BetaHLoss, BtcvaeLoss, EntropyLoss, PearsonLoss, TotalVariationLoss
from lfxai.models.pretext import Identity, Mask, RandomNoise
from lfxai.utils.datasets import MaskedMNIST
from lfxai.utils.feature_attribution import generate_masks
from lfxai.utils.metrics import (
    compute_metrics,
    cos_saliency,
    count_activated_neurons,
    entropy_saliency_tensor,
    entropy_saliency,
    pearson_saliency,
    similarity_rates,
    spearman_saliency,
    pearson_saliency_tensor
)
from lfxai.utils.visualize import (
    correlation_latex_table,
    plot_pretext_saliencies,
    plot_pretext_top_example,
    plot_vae_saliencies,
    vae_box_plots,
)

In [ ]:
W=32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
data_dir = Path.cwd() / "data/mnist"

img_size = (1, W, W)
encoder = EncoderBurgess(img_size, 3)

baseline_image = torch.zeros((1, 1, W, W), device=device)
test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)


test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
test_dataset.transform = test_transform
test_dataset.data, test_dataset.targets = test_dataset.data[[1,2,3,4,5]], test_dataset.targets[[1,2,3,4,5]]
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False)

gradshap = GradientShap(encoder.mu)

In [ ]:
saliency_tensor = tensor_attribution(encoder.mu, 3, test_loader, device, gradshap, baseline_image)
saliency_array = attribute_individual_dim(encoder.mu, 3, test_loader, device, gradshap, baseline_image)

In [ ]:
def disvae_feature_importance(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 5,
    dim_latent: int = 3,
    n_epochs: int = 50,
    beta_list: list = [1],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [BetaHLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Beta"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for beta, loss, run in itertools.product(
        beta_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.beta = beta
        name = f"{str(loss)}-vae_beta{beta}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), beta] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [ ]:
disvae_feature_importance()

Creating saving directory /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/vae
Creating metrics csv in /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/vae/metrics.csv
Now fitting Beta-vae_beta1_run1


Epoch 1/50 	 
Train loss 303 	 Test loss 256 	 


Epoch 2/50 	 
Train loss 251 	 Test loss 247 	 


Epoch 3/50 	 
Train loss 245 	 Test loss 243 	 


Epoch 4/50 	 
Train loss 232 	 Test loss 224 	 


...

Epoch 46/50 	 
Train loss 177 	 Test loss 178 	 


Epoch 47/50 	 
Train loss 177 	 Test loss 178 	 


Epoch 48/50 	 
Train loss 176 	 Test loss 179 	 


Epoch 49/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 50/50 	 
Train loss 176 	 Test loss 178 	 
Model Beta-vae_beta1_run5 	 Pearson Correlation 0.26	Entropy 0.71	Active Neurons 1.3


In [6]:
 def entropy_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 1,
    dim_latent: int = 3,
    n_epochs: int = 50,
    alpha_list: list = [0.5],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    #train_dataset.data, train_dataset.targets = train_dataset.data[[i for i in range(50)]], train_dataset.targets[[i for i in range(50)]]
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    #test_dataset.data, test_dataset.targets = test_dataset.data[[i for i in range(50)]], test_dataset.targets[[i for i in range(50)]]
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/entropy_vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [EntropyLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [5]:
entropy_vae(alpha_list=[0.5])

Creating saving directory /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/entropy_vae
Creating metrics csv in /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/entropy_vae/metrics.csv
Now fitting Entropy-vae_alpha0.5_run1


Epoch 1/50 	 
Train loss 305 	 Test loss 256 	 


Epoch 2/50 	 
Train loss 252 	 Test loss 249 	 


Epoch 3/50 	 
Train loss 242 	 Test loss 250 	 


Epoch 4/50 	 
Train loss 232 	 Test loss 253 	 


...

Epoch 6/50 	 
Train loss 228 	 Test loss 253 	 


Epoch 7/50 	 
Train loss 223 	 Test loss 228 	 


Epoch 8/50 	 
Train loss 217 	 Test loss 214 	 


Epoch 9/50 	 
Train loss 206 	 Test loss 199 	 


KeyboardInterrupt: ignored

In [7]:
entropy_vae(alpha_list=[1])

Now fitting Entropy-vae_alpha1_run1


Epoch 1/50 	 
Train loss 307 	 Test loss 258 	 


Epoch 2/50 	 
Train loss 252 	 Test loss 248 	 


Epoch 3/50 	 
Train loss 241 	 Test loss 250 	 


Epoch 4/50 	 
Train loss 231 	 Test loss 255 	 


...

Epoch 8/50 	 
Train loss 217 	 Test loss 257 	 


Epoch 9/50 	 
Train loss 216 	 Test loss 252 	 


Epoch 10/50 	 
Train loss 214 	 Test loss 253 	 


Epoch 11/50 	 
Train loss 213 	 Test loss 255 	 


Epoch 12/50 	 
Train loss 212 	 Test loss 255 	 
Model Entropy-vae_alpha1_run1 	 Pearson Correlation 0.47	Entropy 0.029	Active Neurons 1


In [8]:
entropy_vae(alpha_list=[5])

Now fitting Entropy-vae_alpha5_run1


Epoch 1/50 	 
Train loss 312 	 Test loss 273 	 


Epoch 2/50 	 
Train loss 273 	 Test loss 272 	 


Epoch 3/50 	 
Train loss 273 	 Test loss 272 	 


Epoch 4/50 	 
Train loss 272 	 Test loss 272 	 


...

Epoch 22/50 	 
Train loss 212 	 Test loss 259 	 


Epoch 23/50 	 
Train loss 210 	 Test loss 256 	 


Epoch 24/50 	 
Train loss 210 	 Test loss 262 	 


Epoch 25/50 	 
Train loss 209 	 Test loss 258 	 


Epoch 26/50 	 
Train loss 208 	 Test loss 270 	 
Model Entropy-vae_alpha5_run1 	 Pearson Correlation 0.44	Entropy 0.015	Active Neurons 1


In [ ]:
entropy_vae(alpha_list=[10])

In [ ]:
def pearson_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 4,
    dim_latent: int = 3,
    n_epochs: int = 50,
    alpha_list: list = [5],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    #train_dataset.data, train_dataset.targets = train_dataset.data[[i for i in range(50)]], train_dataset.targets[[i for i in range(50)]]
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    #test_dataset.data, test_dataset.targets = test_dataset.data[[i for i in range(50)]], test_dataset.targets[[i for i in range(50)]]
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/pearson_vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [PearsonLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [ ]:
pearson_vae(alpha_list=[1])

Creating saving directory /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/pearson_vae
Creating metrics csv in /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/pearson_vae/metrics.csv
Now fitting Pearson-vae_alpha1_run1


Epoch 1/50 	 
Train loss 304 	 Test loss 256 	 


Epoch 2/50 	 
Train loss 251 	 Test loss 248 	 


Epoch 3/50 	 
Train loss 237 	 Test loss 216 	 


Epoch 4/50 	 
Train loss 207 	 Test loss 202 	 


...

Epoch 46/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 47/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 48/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 49/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 50/50 	 
Train loss 175 	 Test loss 177 	 
Model Pearson-vae_alpha1_run4 	 Pearson Correlation 0.21	Entropy 0.68	Active Neurons 1.3


In [ ]:
pearson_vae(alpha_list = [5])

Now fitting Pearson-vae_alpha5_run1


Epoch 1/50 	 
Train loss 302 	 Test loss 255 	 


Epoch 2/50 	 
Train loss 246 	 Test loss 224 	 


Epoch 3/50 	 
Train loss 212 	 Test loss 204 	 


Epoch 4/50 	 
Train loss 201 	 Test loss 198 	 


...

Epoch 46/50 	 
Train loss 175 	 Test loss 177 	 


Epoch 47/50 	 
Train loss 175 	 Test loss 177 	 


Epoch 48/50 	 
Train loss 175 	 Test loss 177 	 


Epoch 49/50 	 
Train loss 175 	 Test loss 177 	 


Epoch 50/50 	 
Train loss 175 	 Test loss 177 	 
Model Pearson-vae_alpha5_run4 	 Pearson Correlation 0.13	Entropy 0.66	Active Neurons 1.2


In [ ]:
pearson_vae(alpha_list = [10])

Now fitting Pearson-vae_alpha10_run1


Epoch 1/50 	 
Train loss 301 	 Test loss 246 	 


Epoch 2/50 	 
Train loss 234 	 Test loss 227 	 


Epoch 3/50 	 
Train loss 216 	 Test loss 209 	 


Epoch 4/50 	 
Train loss 204 	 Test loss 201 	 


...

Epoch 46/50 	 
Train loss 176 	 Test loss 179 	 


Epoch 47/50 	 
Train loss 177 	 Test loss 178 	 


Epoch 48/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 49/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 50/50 	 
Train loss 176 	 Test loss 178 	 
Model Pearson-vae_alpha10_run4 	 Pearson Correlation 0.073	Entropy 0.62	Active Neurons 1.2


In [3]:
def total_variation_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 1,
    dim_latent: int = 3,
    n_epochs: int = 50,
    alpha_list: list = [5],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    #train_dataset.data, train_dataset.targets = train_dataset.data[[i for i in range(50)]], train_dataset.targets[[i for i in range(50)]]
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    #test_dataset.data, test_dataset.targets = test_dataset.data[[i for i in range(50)]], test_dataset.targets[[i for i in range(50)]]
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/test"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [TotalVariationLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [4]:
total_variation_vae(alpha_list=[5])

Now fitting TotalVariation-vae_alpha5_run1


Epoch 1/50 	 
Train loss 311 	 Test loss 276 	 


Epoch 2/50 	 
Train loss 261 	 Test loss 297 	 


Epoch 3/50 	 
Train loss 256 	 Test loss 316 	 


Epoch 4/50 	 
Train loss 255 	 Test loss 337 	 


...

Epoch 7/50 	 
Train loss 252 	 Test loss 345 	 


Epoch 8/50 	 
Train loss 251 	 Test loss 362 	 


Epoch 9/50 	 
Train loss 251 	 Test loss 349 	 


Epoch 10/50 	 
Train loss 250 	 Test loss 363 	 


Epoch 11/50 	 
Train loss 250 	 Test loss 361 	 
Model TotalVariation-vae_alpha5_run1 	 Pearson Correlation 0.35	Entropy 0.68	Active Neurons 1.2


In [5]:
total_variation_vae(alpha_list=[1])

Now fitting TotalVariation-vae_alpha1_run1


Epoch 1/50 	 
Train loss 306 	 Test loss 287 	 


Epoch 2/50 	 
Train loss 258 	 Test loss 301 	 


Epoch 3/50 	 
Train loss 256 	 Test loss 321 	 


Epoch 4/50 	 
Train loss 254 	 Test loss 338 	 


...

Epoch 7/50 	 
Train loss 252 	 Test loss 340 	 


Epoch 8/50 	 
Train loss 251 	 Test loss 354 	 


Epoch 9/50 	 
Train loss 251 	 Test loss 354 	 


Epoch 10/50 	 
Train loss 250 	 Test loss 378 	 


Epoch 11/50 	 
Train loss 249 	 Test loss 356 	 
Model TotalVariation-vae_alpha1_run1 	 Pearson Correlation 0.44	Entropy 0.71	Active Neurons 1.3


In [ ]:
total_variation_vae(alpha_list=[0.5])

Now fitting TotalVariation-vae_alpha0.5_run1


 39%|███▉      | 78/200 [00:14<00:20,  5.94batches/s]

In [ ]:
total_variation_vae(alpha_list=[1000])

Now fitting TotalVariation-vae_alpha1000_run1


Epoch 1/50 	 
Train loss 304 	 Test loss 284 	 


Epoch 2/50 	 
Train loss 258 	 Test loss 301 	 


Epoch 3/50 	 
Train loss 255 	 Test loss 315 	 


Epoch 4/50 	 
Train loss 254 	 Test loss 320 	 


...

Epoch 46/50 	 
Train loss 190 	 Test loss 192 	 


Epoch 47/50 	 
Train loss 190 	 Test loss 191 	 


Epoch 48/50 	 
Train loss 190 	 Test loss 193 	 


Epoch 49/50 	 
Train loss 190 	 Test loss 191 	 


Epoch 50/50 	 
Train loss 190 	 Test loss 191 	 
Model TotalVariation-vae_alpha1000_run1 	 Pearson Correlation 0.31	Entropy 0.73	Active Neurons 1.3
